In [1239]:
import os
import time
import tqdm
import copy
import bvbabel
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import BVClient
bvc = BVClient(port=5555)

# Convert VMP to SMP

In [ ]:
hemisphere = 'LH'
region = 'A1'

subject_list = ['S01']


conditions = ['CondA', 'CondB']

for subject in subject_list:
    for condition in conditions:

        bv.close_all()

        # load vmr
        print(f'openning subject {subject}')
        vmr_filename = glob(f'/path/to/{subject}_*_{hemisphere}.vmr')[0]
        doc = bv.open(vmr_filename)
        
        # load vmp
        print(f'loading vmp for condition {condition}...')
        vmp_filename = f'/path/to/{subject}_{hemisphere}_{region}_{condition}.vmp'
        doc.load_maps(vmp_filename)
        doc.show_map(0)
        
        for depth_mesh in glob(f'/path/to/{subject}_*_HIRES_SPH_equi-vol_D*.srf'):
            # load srf
            print(f'openning surface mesh {depth_mesh}')
            doc.load_mesh(depth_mesh)
            
            # check if file was created
            mesh_map_prefix = depth_mesh.split('_')[-1].split('srf')[0][:-1]
            smp_filename = f'/path/to/{subject}_{hemisphere}_{region}_{condition}_depth_{mesh_map_prefix}.smp'
            if os.path.exists(smp_filename):
                print(f'file {mesh_map_prefix} alread exists, skipping...')
                continue
                

            print(f'creating map...')
            mesh = doc.current_mesh
            interpolation_method=2
            #sample_only_nonzero_values=True
            sample_only_nonzero_values=False

            mesh.create_map_from_volume_map(interpolation_method, sample_only_nonzero_values)
            
            # save smp
            print(f'saving smp as {smp_filename}')
            mesh.save_maps(smp_filename)
            mesh.close()
            del mesh
        # save state for finished subject/condition
        doc.close()
        del doc

## Combine smps for single trial beta map

In [ ]:
hemisphere = 'LH'
region = 'FOP3'

subject = 'S04'
condition = 'WhatOn_Congruent_PA'

# unload all
bv.close_all()

# load vmr
print(f'openning subject {subject}')
vmr_filename = glob(f'/path/to/{subject}_*_{hemisphere}.vmr')[0]
doc = bv.open(vmr_filename)

# load vmp
print(f'loading vmp for condition {condition}...')
vmp_filename = f'/path/to.vmp'
doc.load_maps(vmp_filename)
doc.show_map(0)

for depth_mesh in glob(f'/path/to/{subject}_*_HIRES_SPH_equi-vol_D*.srf'):
    # load srf
    print(f'openning surface mesh {depth_mesh}')
    doc.load_mesh(depth_mesh)
    
    # check if file was created
    mesh_map_prefix = depth_mesh.split('_')[-1].split('srf')[0][:-1]
    smp_filename = f'/path/to/{subject}_{hemisphere}_{region}_{condition}_depth_{mesh_map_prefix}_interpol1.smp'
    if os.path.exists(smp_filename):
        print(f'file {mesh_map_prefix} alread exists, skipping...')
        continue
        

    print(f'creating map...')
    mesh = doc.current_mesh
    interpolation_method=1
    #sample_only_nonzero_values=True
    sample_only_nonzero_values=False

    mesh.create_map_from_volume_map(interpolation_method, sample_only_nonzero_values)
    
    # save smp
    print(f'saving smp as {smp_filename}')
    mesh.save_maps(smp_filename)
    del mesh
# save state for finished subject/condition
doc.close()
del doc

In [ ]:
mesh_map_prefixes = []
headers = []
datas = []
for i in range(1, 12):
    filename = glob(f'/path/to/depth_D-{i}-*_interpol1.smp')[0]
    print(f'loading {filename}')
    head, data = bvbabel.smp.read_smp(filename)
    headers.append(head)
    datas.append(data)
    mesh_map_prefixes.append(filename.split('_')[-1].split('srf')[0][:-4])

In [ ]:
data_combined = np.hstack(datas)
head_combined = copy.deepcopy(headers[0])

comb_names = []
for head, prefix in zip(headers, mesh_map_prefixes):
    for smap in head['Map']:
        comb_names.append(prefix + '_' + smap['Name'])

head_combined['Nr maps'] = data_combined.shape[1]
head_combined['SRF file'] = '/path/to/sub_Mid-GM_BL2_BL2_RECOSM_D164k_HIRES_SPH.srf'

head_combined['Map'] = []
for cn in comb_names:
    entry = {
        'Map type': 1, 
        'Cluster size': 25, 
        'Cluster checkbox': 0, 
        'Threshold min': 1.649999976158142, 
        'Threshold max': 8.0, 
        'Threshold include greater than max': 1, 
        'Degrees of freedom 1': 2435, 
        'Degrees of freedom 2': 1, 
        'Show positive negative': 3, 
        'Bonferroni correction value': 45555, 
        'RGB positive min': np.array([255,   0,   0], dtype=np.uint8), 
        'RGB positive max': np.array([255, 255,   0], dtype=np.uint8), 
        'RGB negative min': np.array([255,   0, 255], dtype=np.uint8), 
        'RGB negative max': np.array([  0,   0, 255], dtype=np.uint8), 
        'RGB or LUT': 0, 
        'LUT file': '<default>', 
        'Color transparency': 1.0, 
        'Name': cn}
    
    head_combined['Map'].append(entry)
    
len(head_combined['Map'])

bvbabel.smp.write_smp('/path/to/single_trial_combined_interpol1.smp', head_combined, data_combined)

# Combine the smps into one (as a checkup)

In [1052]:
full_head, full_data = bvbabel.smp.read_smp('/path/to/sub_11-DepthLevels.smp')

In [ ]:
mesh_map_prefixes = []
headers = []
datas = []
for i in range(1, 12):
    filename = glob(f'/path/to/sub_D-{i}-*.smp')[0]
    filename = glob(f'/path/to/sub_D-{i}-*False.smp')[0]
    print(f'loading {filename}')
    head, data = bvbabel.smp.read_smp(filename)
    headers.append(head)
    datas.append(data)
    
    mesh_map_prefixes.append(filename.split('_')[-1].split('srf')[0][:-4])

In [ ]:
data_combined = np.hstack(datas)
head_combined = copy.deepcopy(headers[0])

comb_names = []
for head, prefix in zip(headers, mesh_map_prefixes):
    for smap in head['Map']:
        comb_names.append(prefix + '_' + smap['Name'])

head_combined['Nr maps'] = data_combined.shape[1]
head_combined['SRF file'] = '/path/to/sub_Mid-GM_BL2_BL2_RECOSM_D164k_HIRES_SPH.srf'

head_combined['Map'] = []
for cn in comb_names:
    entry = {
        'Map type': 1, 
        'Cluster size': 25, 
        'Cluster checkbox': 0, 
        'Threshold min': 1.649999976158142, 
        'Threshold max': 8.0, 
        'Threshold include greater than max': 1, 
        'Degrees of freedom 1': 2435, 
        'Degrees of freedom 2': 1, 
        'Show positive negative': 3, 
        'Bonferroni correction value': 45555, 
        'RGB positive min': np.array([255,   0,   0], dtype=np.uint8), 
        'RGB positive max': np.array([255, 255,   0], dtype=np.uint8), 
        'RGB negative min': np.array([255,   0, 255], dtype=np.uint8), 
        'RGB negative max': np.array([  0,   0, 255], dtype=np.uint8), 
        'RGB or LUT': 0, 
        'LUT file': '<default>', 
        'Color transparency': 1.0, 
        'Name': cn}
    
    head_combined['Map'].append(entry)
    
len(head_combined['Map'])

# bvbabel.smp.write_smp('/filename.smp', head_combined, data_combined)